In [41]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from xfem import *
from xfem.mlset import *
from math import pi,e
from numpy import linspace


In [45]:
aperture = 0.4 # width
H = 0.5 # height
outer = Rectangle(1, 1).Face()
outer.edges.name="outer"
# outer.edges.Max(X).name = "r"
# outer.edges.Min(X).name = "l"
# outer.edges.Min(Y).name = "b"
# outer.edges.Max(Y).name = "t"

Omegaf = MoveTo(0.5-aperture/2, 0.5 - H/2).Rectangle(aperture, H).Face()
Omegaf.edges.name="interface"
Omegap = outer - Omegaf

Omegaf.faces.name="Omegaf"
Omegaf.faces.col = (1, 0, 0)
Omegap.faces.name="Omegap"

geo = Glue([Omegaf, Omegap])
# Draw(geo);

In [46]:
mh = 1/8
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=mh))
Draw(mesh);
# print(mesh.GetBoundaries())
# print(mesh.GetMaterials())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [47]:
# center point
a = 0.5
b = 0.5

line1 = x - (a+aperture/2)
line2 = x - (a-aperture/2)
line3 = y - (b+H/2)
line4 = y - (b-H/2)
level_sets = (line1, line2, line3, line4)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(lsets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
    # DrawDC(lset_p1,-1,1,mesh)
domain = DomainTypeArray((NEG,POS,NEG,POS)) 
DrawDC = MakeDiscontinuousDraw(Draw)
DrawDC(domain.Indicator(lsets_p1),-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [48]:
# 交界面的水平集函数
a = 0.5
b = 0.5
phi1 = IfPos(x-a, x-a-aperture/2, a-x-aperture/2)
phi2 = IfPos(y-b, y-b-H/2, b-y-H/2)
levelset = IfPos(phi2-phi1, phi2, phi1)
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
ci = CutInfo(mesh,lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)
gamma_facets = GetFacetsWithNeighborTypes(mesh, a=ci.GetElementsOfType(HASNEG), b=ci.GetElementsOfType(POS),bnd_val_a=False,bnd_val_b=False,use_and=True)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,gamma_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")  

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [49]:
# penalty parameters
order_eta = 2
order_pp = 1
order_pf = 1

beta_eta = 100
beta_pp = 1e2
beta_pf = 1e2
sigma = 200
# physical parameters
mu  = 10
lam = 100
alpha = 1
Kp = 1
Kf = 1

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [50]:
# 定义解析解
eta_x = sin(pi*x)*cos(pi*y)
eta_y = cos(pi*x)*sin(pi*y)
exact_eta = CF((eta_x,eta_y))
exact_p = sin(pi*x)*sin(pi*y)

# strain tensor
epsilon_xx = eta_x.Diff(x)
epsilon_yy = eta_y.Diff(y) 
epsilon_xy = 0.5*(eta_x.Diff(y) +  eta_y.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy

# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

# 向量形式
fe = CF( (f_x,f_y) )
fp = -Kp*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y))
ff = -Kf*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y))

etaD = exact_eta
pD = exact_p

In [51]:
# DG spaces
Eh = VectorL2(mesh, definedon="Omegap", order=order_eta, dirichlet=".*", dgjumps=True) # space for velocity
Ph = L2(mesh, definedon="Omegap", order=order_pp, dirichlet=".*", dgjumps=True) # space for pressure in poroelasiticity
Fh = L2(mesh, definedon="Omegaf", order=order_pf, dirichlet=".*", dgjumps=True) # space for pressure in poroelasiticity
E = Compress(Eh)
P = Compress(Ph)
F = Compress(Fh)
fes = E*P*F
(eta,pp,pf), (xi,qp,qf) = fes.TnT()

# define integral operators
dOmegap = dx(definedon=mesh.Materials("Omegap")) # integral on Omegap
dOmegaf = dx(definedon=mesh.Materials("Omegaf")) # integral on Omegaf
dso = ds(skeleton=True, definedon=mesh.Boundaries("outer")) # integral on outer faces
dgamma = ds(skeleton=True, definedon=mesh.Boundaries("interface")) # integral on interfaces

dsip = dx(skeleton=True, definedon=mesh.Materials("Omegap")) # integral on interior faces of Omegap
dsif = dx(skeleton=True, definedon=mesh.Materials("Omegaf")) # integral on interior faces of Omegaf


In [52]:
##### test for dsif/dsip integral operator
# p,q = F.TnT()
# ah = BilinearForm(F)
# jump_p = p - p.Other()
# jump_q = q - q.Other()
# # ah += p*q*dOmegaf
# ah += jump_p * jump_q * dsif
# ah.Assemble()
# print(ah.mat)

In [53]:
# test for integral operators
f1 = CoefficientFunction(1)
print(Integrate(f1*dg, mesh))

0.18093372744093267


In [54]:
# Define the jumps and the averages
jump_eta = eta - eta.Other()
jump_xi = xi - xi.Other()
jump_pp = pp - pp.Other()
jump_qp = qp - qp.Other()
jump_pf = pf - pf.Other()
jump_qf = qf - qf.Other()
h = specialcf.mesh_size  
n = specialcf.normal(2)
strain_eta = Sym(Grad(eta))
strain_xi = Sym(Grad(xi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*n
mean_stress_xi = 0.5*(Stress(Sym(Grad(xi)))+Stress(Sym(Grad(xi.Other()))))*n

mean_dppdn = 0.5*Kp*(grad(pp)+grad(pp.Other()))*n
mean_dqpdn = 0.5*Kp*(grad(qp)+grad(qp.Other()))*n
mean_dpfdn = 0.5*Kf*(grad(pf)+grad(pf.Other()))*n
mean_dqfdn = 0.5*Kf*(grad(qf)+grad(qf.Other()))*n

mean_pp = 0.5*(pp + pp.Other())
mean_qp = 0.5*(qp + qp.Other())
 

In [55]:
# dg = dx(skeleton=True, definedon=mesh.Materials("Omegaf|Omegap"))

In [57]:
dg = dx(skeleton=True, definedonelements=gamma_facets)
# dg = ds(definedon=mesh.Boundaries("interface")) # integral on interfaces
p1,q1 = Ph.TnT()
p2,q2 = Fh.TnT()
mh = BilinearForm(Fh)
mh += p2*q2*dg 
mh.Assemble()
print(mh.mat)

Row 0:   0: 0.182764   1: -0.182764   2: 0   49: 0   50: 0   51: 0
Row 1:   0: -0.182764   1: 0.182764   2: 0   49: 0   50: 0   51: 0
Row 2:   0: 0   1: 0   2: 0.0609214   49: 0   50: 0   51: 0
Row 3:   3: 0.133333   4: 0.0666667   5: 0.0666667   9: 0   10: 0   11: 0   49: 0   50: 0   51: 0
Row 4:   3: 0.0666667   4: 0.133333   5: 3.46945e-18   9: 0   10: 0   11: 0   49: 0   50: 0   51: 0
Row 5:   3: 0.0666667   4: 3.46945e-18   5: 0.0444444   9: 0   10: 0   11: 0   49: 0   50: 0   51: 0
Row 6:   6: 0.133333   7: 0.0666667   8: 0.0666667   9: 0   10: 0   11: 0   12: 0   13: 0   14: 0
Row 7:   6: 0.0666667   7: 0.133333   8: 3.46945e-18   9: 0   10: 0   11: 0   12: 0   13: 0   14: 0
Row 8:   6: 0.0666667   7: 3.46945e-18   8: 0.0444444   9: 0   10: 0   11: 0   12: 0   13: 0   14: 0
Row 9:   3: 0   4: 0   5: 0   6: 0   7: 0   8: 0   9: 0   10: 0   11: 0   59: 0   60: 0   61: 0
Row 10:   3: 0   4: 0   5: 0   6: 0   7: 0   8: 0   9: 0   10: 0   11: 0   59: 0   60: 0   61: 0
Row 11:   3: 0 

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


In [ ]:
sh = BilinearForm(F)
sh += p2*q2*dgamma
sh.Assemble()
print(sh.mat)

In [ ]:
ah = BilinearForm(fes)
# Ae
ah += 2*mu*InnerProduct(strain_eta,strain_xi)*dOmegap + lam*div(eta)*div(xi)*dOmegap \
        - (InnerProduct(mean_stress_eta,jump_xi) + InnerProduct(mean_stress_xi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_xi))*dsip \
        - (InnerProduct(Stress(Sym(Grad(eta)))*n,xi) + InnerProduct(Stress(Sym(Grad(xi)))*n,eta) - beta_eta/h*InnerProduct(eta,xi))*dso
# Be
ah += -alpha*(div(xi)*pp*dOmegap - mean_pp*jump_xi*n*dsip - pp*xi*n*dso)
# I
ah += pf*n*xi*dgamma

# Ap
# ah += Kp*grad(pp)*grad(qp)*dOmegap \
#         - (mean_dppdn*jump_qp + mean_dqpdn*jump_pp - beta_pp/h*jump_pp*jump_qp)*dsip \
#         - (Kp*grad(pp)*n*qp + Kp*grad(qp)*n*pp - beta_pp/h*pp*qp)*dso \
#         - Kp*grad(pp)*n*qp*dgamma
ah += Kp*grad(pp)*grad(qp)*dOmegap \
        - (mean_dppdn*jump_qp + mean_dqpdn*jump_pp - beta_pp/h*jump_pp*jump_qp)*dsip \
        - (Kp*grad(pp)*n*qp + Kp*grad(qp)*n*pp - beta_pp/h*pp*qp)*dso

# Af
# ah += Kf*grad(pf)*grad(qf)*dOmegaf \
#         - (mean_dpfdn*jump_qf + mean_dqfdn*jump_pf - beta_pf/h*jump_pf*jump_qf)*dsif \
#         - Kf*grad(pf)*n*qf*dgamma
ah += Kf*grad(pf)*grad(qf)*dOmegaf \
        - (mean_dpfdn*jump_qf + mean_dqfdn*jump_pf - beta_pf/h*jump_pf*jump_qf)*dsif 

ah += - 0.5*(Kp*grad(pp) + Kf*grad(pf))*n * (qp - qf) * dgamma
ah += - 0.5*(Kp*grad(qp) + Kf*grad(qf))*n * (pp - pf) * dgamma # 对称项

ah += sigma/h * (pp-pf)*(qp - qf) * dgamma # continuity for pressure

ah.Assemble()

In [ ]:
# r.h.s
fh = LinearForm(fes)
fh += fe*xi*dOmegap - InnerProduct(etaD,Stress(Sym(Grad(xi)))*n)*dso + beta_eta/h*etaD*xi*dso
fh += fp*qp*dOmegap - Kp*grad(qp)*n*pD*dso + beta_pp/h*pD*qp*dso
fh += ff*qf*dOmegaf

fh.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = ah.mat.Inverse() * fh.vec

In [ ]:
error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2*dOmegap, mesh))
print(error_eta)
error_pp = sqrt(Integrate((gfu.components[1] - exact_p)**2*dOmegap, mesh))
print(error_pp)
error_pf = sqrt(Integrate((gfu.components[2] - exact_p)**2*dOmegaf, mesh))
print(error_pf)

In [ ]:
Draw(gfu.components[0],mesh,min=0,max=1)
exact_etah = GridFunction(E)
exact_etah.Set(exact_eta)
Draw(exact_etah,mesh)

In [ ]:
Draw(gfu.components[1],mesh)
exact_pp = GridFunction(P)
exact_pp.Set(exact_p)
Draw(exact_pp,mesh)

In [ ]:
Draw(gfu.components[2],mesh)
exact_pf = GridFunction(F)
exact_pf.Set(exact_p)
Draw(exact_pf,mesh)